# Benchmark Runner
After the prep work is done you can run the benchmark. First, some basic configuration work.

In [ ]:
from remotemanager.dataset import Dataset
from time import sleep
from os.path import join

computer = "fugaku"
computer = "hokusai"

indir = "inp"
natoms = 54  # number of atoms in a molecule

# Small System to Compute
geom = "2CzPN_2"

# Large Systems to Compute
geoms = ["2CzPN_14", "2CzPN_18", "2CzPN_22", "2CzPN_26",
         "2CzPN_30", "2CzPN_34", "2CzPN_38", "2CzPN_42",
         "2CzPN_50", "2CzPN_58", "2CzPN_66", "2CzPN_74",
         "2CzPN_82", "2CzPN_90"]

In [ ]:
%load_ext remotemanager

Setup your computer.

In [ ]:
if computer == "hokusai":
    from remotecomputer.hokusai import Hokusai
    url = Hokusai()
    url.key = "Q22460"
    url.conda = "remote"
    # Default mpi/omp
    url.mpi = 1
    url.omp = 1
    url.elapse = "0:30:00"
    url.path_to_bigdft = "/home/wddawson/binaries/bdft"
    remote_dir = join("runs", "2023", "benchmark-tn")
    
    # For scanning the MPI/OMP choices
    mpi_omp = [(1, 40), (2, 20), (4, 10), (8, 5), (5, 8)]
    
    # Parameters for the large calculations
    large_mpi = 64
    large_omp = 10
    large_elapse = "4:00:00"
elif computer == "fugaku":
    from remotecomputer.fugaku import Fugaku
    url = Fugaku(user="u10035")
    url.key = "ra000009"
    url.mpi = 1
    url.omp = 1
    url.elapse = "00:30:00"
    url.path_to_bigdft = join("/home", url.key.value, "data", 
                              url.user, "binaries", "bdft")
    remote_dir = join("/home", url.key.value, "data",
                      url.user, "runs", "2023", "benchmark-tn")

    # For scanning the MPI/OMP choices
    mpi_omp = [(1, 48), (2, 24), (4, 12), (8, 6), (12, 4)]
    
    # Parameters for the large calculations
    large_mpi = 1024
    large_omp = 6
    large_elapse = "1:00:00"

A routine that runs a calculation using the correct parameters for benchmarking.

In [ ]:
def treat_system(sname, mpi, omp, indir=indir, linear=False):
    from BigDFT.IO import read_pdb
    from BigDFT.Inputfiles import Inputfile
    from BigDFT.Calculators import SystemCalculator
    from os.path import join
    
    # System
    with open(join(indir, sname + ".pdb")) as ifile:
        sys = read_pdb(ifile)
    
    # Input
    inp = Inputfile()
    inp.set_xc("PBE")
    inp.set_hgrid(0.5)
    inp.set_rmult(coarse=5.0, fine=7.0)
    inp.set_psp_nlcc()
    
    if linear:
        inp["import"] = "linear"
        inp["perf"] = {"check_sumrho": 0, "check_overlap": 0}
        inp["lin_general"] = {"charge_multipoles": 0, "output_mat": 0}
    
    # Compute
    calc = SystemCalculator(skip=True)
    
    # Determine the run name
    run_name = sname + "_" + str(mpi) + "_" + str(omp)
    log = calc.run(sys=sys, input=inp, name=run_name)

## Cubic Single Node Performance
The first benchmark checks the single node performance using a few combinations of threads and mpi ranks.

In [ ]:
single_ds = Dataset(function = treat_system,
                    url = url, extra_files_send=[indir],
                    dbfile="single_db_" + computer)

Add runs with each parameter.

In [ ]:
for comb in mpi_omp:
    args = {"sname": geom, "mpi": comb[0], "omp": comb[1]}
    single_ds.append_run(arguments = args, mpi=comb[0], omp=comb[1],
                         remote_dir=remote_dir)

In [ ]:
single_ds.run()

In [ ]:
while not single_ds.all_finished:
    sleep(120)

Fetch the time to solution data.

In [ ]:
url.mpi = 1
url.omp = 1
url.elapse = "0:10:00"
db = "single_fetch_" + computer

In [ ]:
%%execute_remotely url=url remote_dir=remote_dir dbfile=db : mpi_omp=mpi_omp sname=geom
from yaml import load, SafeLoader
data = {}
for r in mpi_omp:
    run_name = sname + "_" + str(r[0]) + "_" + str(r[1])
    with open("time-" + run_name + ".yaml") as ifile:
        full = load(ifile, Loader=SafeLoader)
    data[run_name] = {"time": full["WFN_OPT"]["Classes"]["Total"][1]}

return data

In [ ]:
magic_dataset.fetch_results()
times = magic_dataset.results[0]

Plot.

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(1, 1, figsize=(4, 3))
axs.plot([x["time"] for x in times.values()] , 'kx', markersize=14)
axs.set_ylabel("Time (s)", fontsize=14)
axs.set_xticks(range(len(list(times))))
axs.set_xticklabels(["MPI:" + str(comb[0]) + 
                     "; OMP:" + str(comb[1]) 
                     for comb in mpi_omp], rotation=90)
pass

## Linear Scaling Calculations
We will now switch to the linear scaling mode. We will run calculations of various size on a reasonable sized partition, and examine the performance.

In [ ]:
LVC_set = Dataset(function = treat_system,
                  url = url, dbfile="scale_db_" + computer)
url.elapse = large_elapse

In [ ]:
for g in geoms:
    args = {"sname": g, "mpi": large_mpi, "omp": large_omp, "linear": True}
    LVC_set.append_run(arguments = args, remote_dir=remote_dir,
                       mpi=large_mpi, omp=large_omp)

In [ ]:
LVC_set.run()

In [ ]:
while not single_ds.all_finished:
    sleep(120)

Fetch the times.

In [ ]:
url.mpi = 1
url.omp = 1
url.elapse = "0:10:00"
db = "scale_fetch_" + computer

In [ ]:
%%execute_remotely url=url remote_dir=remote_dir dbfile=db : geoms=geoms mpi=large_mpi omp=large_omp
from yaml import load, SafeLoader
data = {}
for g in geoms:
    run_name = g + "_" + str(mpi) + "_" + str(omp)
    with open("time-" + run_name + ".yaml") as ifile:
        full = load(ifile, Loader=SafeLoader)
    data[run_name] = {"time": full["WFN_OPT"]["Classes"]["Total"][1]}

return data

In [ ]:
magic_dataset.fetch_results()
times = magic_dataset.results[0]

Plot the time vs. number of atoms.

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(1, 1, figsize=(4, 3))
nats = [int(x.split("_")[-1])*54 for x in geoms]
minutes = [times[x + "_" + str(large_mpi) + 
                 "_" + str(large_omp)]["time"]/60 for x in geoms]
axs.plot([(m/a) * large_mpi*large_omp 
          for m, a in zip(minutes, nats)] , 
         'kx', markersize=14, label="Hokusai")
axs.set_ylabel("CPU Minutes per Atom", fontsize=14)
axs.set_xticks(range(len(list(times))))
axs.set_xticklabels(nats, rotation=90)
axs.set_xlabel("Number of Atoms", fontsize=14)
axs.legend()
fig.tight_layout()
fig.savefig("benchmark.png", dpi=300)
pass